In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])[['Customers']]
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Store', 'Date'])
train_df = train_df.join(df_basic, how='left')
train_df.reset_index(inplace=True)
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,Customers,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,1,2015-07-31,555,5,1,1,0,1,31,4,212,7,3,31,2015
1,2,2015-07-31,625,5,1,1,0,1,31,4,212,7,3,31,2015
2,3,2015-07-31,821,5,1,1,0,1,31,4,212,7,3,31,2015
3,4,2015-07-31,1498,5,1,1,0,1,31,4,212,7,3,31,2015
4,5,2015-07-31,559,5,1,1,0,1,31,4,212,7,3,31,2015


In [4]:
del train_df['Date']

# Big Model

In [6]:
regressor_big = ExtraTreesRegressor(n_estimators=50, n_jobs=-1)
features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
predictions = cross_validation.cross_val_predict(regressor_big, train_df[features], train_df['Customers'])
                                   

In [7]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [8]:
score(predicted=predictions, actual=train_df['Customers'])

0.5761407329039374

In [9]:
train_df['Predictions-big_model'] = predictions

# MultiModel

In [25]:
predictions_dfs = {}

i = 0
features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
for store in train_df['Store'].unique().tolist():
    if (i % 100 == 0):
        print (i)
    i = i + 1
    store_df = train_df[train_df['Store'] == store].copy()
    
    regressor = ExtraTreesRegressor(n_estimators=50, n_jobs=-1)
    predictions = cross_validation.cross_val_predict(regressor, store_df[features], store_df['Customers'])
    store_df['Prediction-mini_model'] = predictions
    predictions_dfs[store] = store_df


0
100
200
300
400
500
600
700
800
900
1000
1100


In [26]:
predictions_df = pandas.concat(predictions_dfs.values())

In [27]:
score(predicted=predictions_df['Prediction-mini_model'], actual=predictions_df['Customers'])

0.5428566910914857